In [ ]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

In [ ]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
infinity = solver.infinity()
# x and y are integer non-negative variables.
wa = solver.IntVar(0.0, infinity, 'warrior')
ri = solver.IntVar(0.0, infinity, 'rider')
sh = solver.IntVar(0.0, infinity, 'shied')

print('Number of variables =', solver.NumVariables())

In [ ]:
# x + 7 * y <= 17.5.
solver.Add(10 * wa + 15 * ri + 15 * sh == 15)

# x <= 3.5.
#solver.Add(x <= 3.5)

print('Number of constraints =', solver.NumConstraints())

In [ ]:
status = solver.Solve()

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('wa =', wa.solution_value())
    print('ri =', ri.solution_value())
    print('sh =', sh.solution_value())
else:
    print('The problem does not have an optimal solution.')

In [ ]:
count = 0
while solver.NextSolution():  # <-- NextSolution() always returns False!
    count += 1
    print("wa =", wa.solution_value(), "ri =", ri.solution_value(), "sh =", sh.solution_value())
print("\nNumber of solutions found:", count)

In [ ]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

x00 = model.NewBoolVar('x00')
x01 = model.NewBoolVar('x01')
x02 = model.NewBoolVar('x02')

model.AddBoolOr([x00, x01, x02.Not()])
model.AddBoolOr([x00.Not(), x02.Not()])

class VarArraySolutionCollector(cp_model.CpSolverSolutionCallback):

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.solution_list = []

    def on_solution_callback(self):
        self.solution_list.append([self.Value(v) for v in self.__variables])

# Create a solver and solve.
solver = cp_model.CpSolver()
solution_collector = VarArraySolutionCollector([x00, x01, x02])
solver.SearchForAllSolutions(model, solution_collector)

solution_collector.solution_list

#Out[33]: [[0, 1, 0], [0, 0, 0], [0, 1, 1], [1, 1, 0], [1, 0, 0]]